<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/es_whisper_small_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-6jemljb8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-6jemljb8
  Resolved https://github.com/huggingface/transformers to commit 2ca73e5ee320078a275e40a95ad32f040a389d39
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8049555 sha256=489229d08945ac05dcd35a0ba14c0fb34f85ce84417e9ccb2781a5bb35d4d092
  Stored in directory: /tmp/pip-ephem-wheel-cache-s5lc88zd/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm


In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "es", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1528
})

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=ee68c762add452a311b094d97063dcd8d14f36790c9474ce329257d201338fc8
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [20]:
model = whisper.load_model("small")

100%|████████████████████████████████████████| 461M/461M [00:03<00:00, 140MiB/s]


In [21]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [22]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [23]:
text

,0
0,señor presidente señor comisario quiero comenz...
1,en el segundo acto del drama llega el consejo ...
2,precisamente en la mayoría de países estamos p...
3,menores no acompañados están sufriendo violenc...
4,por ello pido que la agencia de derechos funda...
...,...
1523,la gente de la mar hace tiempo que viene denun...
1524,el consejo naturalmente conoce los resultados ...
1525,cuándo se crean puestos de trabajo o cuando en...
1526,la unión debe también llevar a cabo una lucha ...


In [24]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [25]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,señor presidente señor comisario quiero comenz...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00067138671875, 0.000274658203125, 0.000671...",16000
1,male,en el segundo acto del drama llega el consejo ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0020751953125, 0.002349853515625, 0.0022583...",16000
2,male,precisamente en la mayoría de países estamos p...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00054931640625, 0.000213623046875, -9.15527...",16000
3,male,menores no acompañados están sufriendo violenc...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000274658203125, -0.000396728515625, -0.00...",16000
4,male,por ello pido que la agencia de derechos funda...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004791259765625, 0.001220703125, 0.00469970...",16000
...,...,...,...,...,...
1523,female,la gente de la mar hace tiempo que viene denun...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001373291015625, -0.001678466796875, -0.00...",16000
1524,female,el consejo naturalmente conoce los resultados ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000152587890625, -6.103515625e-05, 9.15527...",16000
1525,male,cuándo se crean puestos de trabajo o cuando en...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0008544921875, 0.0003662109375, 0.000305175...",16000
1526,male,la unión debe también llevar a cabo una lucha ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000823974609375, 0.000152587890625, -0.0004...",16000


In [26]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,señor presidente señor comisario quiero comenz...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00067138671875, 0.000274658203125, 0.000671...",16000
1,male,en el segundo acto del drama llega el consejo ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0020751953125, 0.002349853515625, 0.0022583...",16000
2,male,precisamente en la mayoría de países estamos p...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00054931640625, 0.000213623046875, -9.15527...",16000
3,male,menores no acompañados están sufriendo violenc...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000274658203125, -0.000396728515625, -0.00...",16000
4,male,por ello pido que la agencia de derechos funda...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004791259765625, 0.001220703125, 0.00469970...",16000
...,...,...,...,...,...
1518,male,además estos acuerdos suponen también un impul...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0008544921875, 0.004241943359375, 0.003204...",16000
1520,male,la pobreza y los déficits democráticos socavan...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00848388671875, -0.001556396484375, 0.0003...",16000
1522,male,tenemos también que garantizar la protección d...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000762939453125, -0.00335693359375, -3.0517...",16000
1525,male,cuándo se crean puestos de trabajo o cuando en...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0008544921875, 0.0003662109375, 0.000305175...",16000


In [27]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
7,female,la inclusión como objetivos prioritarios de la...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00518798828125, -0.002593994140625, -0.002...",16000
11,female,y en lo referente a la inclusión social y a la...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0013427734375, -0.00225830078125, -0.00164...",16000
12,female,los uno seiscientos ochenta y seis millones de...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.003509521484375, -0.00079345703125, -0.0032...",16000
15,female,estamos también de acuerdo con el ponente en q...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0413818359375, -0.012725830078125, 0.00543...",16000
16,female,la oferta de apoyo en materia de reasentamient...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.006378173828125, -0.005889892578125, -0.002...",16000
...,...,...,...,...,...
1519,female,señora presidenta gracias a las ponentes y a l...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000152587890625, -0.000152587890625, -0.000...",16000
1521,female,y lo que está pidiendo a gritos este planeta e...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00146484375, 0.0028076171875, -0.0007324218...",16000
1523,female,la gente de la mar hace tiempo que viene denun...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001373291015625, -0.001678466796875, -0.00...",16000
1524,female,el consejo naturalmente conoce los resultados ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000152587890625, -6.103515625e-05, 9.15527...",16000


In [28]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="es")
    return result["text"]

In [29]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [30]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.30434782608695654
Processed 2 files, WER: 0.3333333333333333
Processed 3 files, WER: 0.19230769230769232
Processed 4 files, WER: 0.09090909090909091
Processed 5 files, WER: 0.15384615384615385
Processed 6 files, WER: 0.5
Processed 7 files, WER: 0.23333333333333334
Processed 8 files, WER: 0.09090909090909091
Processed 9 files, WER: 0.19047619047619047
Processed 10 files, WER: 0.22580645161290322
Processed 11 files, WER: 0.125
Processed 12 files, WER: 0.15384615384615385
Processed 13 files, WER: 0.36666666666666664
Processed 14 files, WER: 0.25
Processed 15 files, WER: 0.19047619047619047
Processed 16 files, WER: 0.15
Processed 17 files, WER: 0.25862068965517243
Processed 18 files, WER: 0.5172413793103449
Processed 19 files, WER: 0.037037037037037035
Processed 20 files, WER: 0.7142857142857143
Processed 21 files, WER: 0.18
Processed 22 files, WER: 0.2318840579710145
Processed 23 files, WER: 0.2
Processed 24 files, WER: 0.1724137931034483
Processed 25 files, WER:

In [19]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.2833333333333333
Processed 2 files, WER: 0.23809523809523808
Processed 3 files, WER: 0.375
Processed 4 files, WER: 0.05555555555555555
Processed 5 files, WER: 0.3055555555555556
Processed 6 files, WER: 0.11538461538461539
Processed 7 files, WER: 0.1875
Processed 8 files, WER: 0.20512820512820512
Processed 9 files, WER: 0.18181818181818182
Processed 10 files, WER: 0.12121212121212122
Processed 11 files, WER: 0.37662337662337664
Processed 12 files, WER: 0.22807017543859648
Processed 13 files, WER: 0.225
Processed 14 files, WER: 0.34210526315789475
Processed 15 files, WER: 0.4
Processed 16 files, WER: 0.20408163265306123
Processed 17 files, WER: 0.05263157894736842
Processed 18 files, WER: 0.2727272727272727
Processed 19 files, WER: 0.12903225806451613
Processed 20 files, WER: 0.21052631578947367
Processed 21 files, WER: 0.3181818181818182
Processed 22 files, WER: 0.14285714285714285
Processed 23 files, WER: 0.08196721311475409
Processed 24 files, WER: 0.375
Proc